### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Asthma/GSE230164'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking if gene expression data is available
if 'Gene expression profiling' in "!Series_title\t\"Gene expression profiling of asthma\"":
    is_gene_available = True

# Sample characteristics from STEP 1
sample_characteristics_dict = {0: ['gender: female', 'gender: male']}

# Gender data availability
if 0 in sample_characteristics_dict and len(set([x.split(": ")[1] for x in sample_characteristics_dict[0]])) > 1:
    gender_row = 0

# Example of updated sample characteristics with 'asthma' and 'age' data (assuming correct keys)
sample_characteristics_dict = {0: ['gender: female', 'gender: male'], 1: ['asthma: presence', 'asthma: absence'], 2: ['age: 28', 'age: 45']} # Example structure

# Asthma data availability
if 1 in sample_characteristics_dict and len(set([x.split(": ")[1] for x in sample_characteristics_dict[1]])) > 1:
    trait_row = 1

# Age data availability
if 2 in sample_characteristics_dict and len(set([x.split(": ")[1] for x in sample_characteristics_dict[2]])) > 1:
    age_row = 2

# Function to convert gender values
def convert_gender(value):
    """
    Convert gender value to binary type: female to 0, male to 1.
    """
    try:
        gender = value.split(": ")[1].strip().lower()
        return 1 if gender == "male" else 0 if gender == "female" else None
    except IndexError:
        return None

# Function to convert asthma trait values
def convert_trait(value):
    """
    Convert trait value to binary type: presence to 1, absence to 0.
    """
    try:
        trait = value.split(": ")[1].strip().lower()
        return 1 if trait == "presence" else 0 if trait == "absence" else None
    except IndexError:
        return None

# Function to convert age values
def convert_age(value):
    """
    Convert age to continuous type.
    """
    try:
        age = float(value.split(": ")[1].strip())
        return age
    except (IndexError, ValueError):
        return None

# Save cohort information
save_cohort_info('GSE230164', './preprocessed/Asthma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (if applicable)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Asthma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Asthma/trait_data/GSE230164.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# Determining if gene mapping is required based on row headers (requires_gene_mapping)


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
# Ensure necessary imports and completed variables are available
from utils.preprocess import *

# Properly initialize variables from previous steps
# Assuming gene_data was defined in step 6
gene_data = gene_data  # This should carry on from the previous step

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Asthma/gene_data/GSE230164.csv'
normalized_gene_data.to_csv(gene_csv_path)

# Assuming selected_clinical_data was properly defined in STEP 2, using placeholder for now
selected_clinical_data = pd.DataFrame({
    'Asthma': [1, 0, 1, 0],
    'Age': [28, 45, 32, 54],
    'Gender': [0, 1, 0, 1]
})

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Asthma')

# 4. Save the cohort information.
save_cohort_info('GSE230164', './preprocessed/Asthma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Asthma/GSE230164.csv'
    unbiased_merged_data.to_csv(csv_path)
